# Tools

In [ ]:
from app import Application
from array import array
from blocks import transform
from blocks.bbs import BBS
from blocks.acm import ACM
from blocks.rt import RT
from errors import ValidationError
from math import gcd
from PIL import Image
from suplementary import number_theory as nt
from time import time

import numpy as np
import sys


primes = array('I')
a = open('primes/100000.txt', 'r')
lines = a.readlines()
for line in lines[5:]:
    for p in line[:-2].split():
        p = int(p)
        if p % 4 == 3 and p < 65536:
            primes.append(int(p))

def pick_random_seed(m):
    while True:
        ret = np.random.randint(1, m)
        if gcd(ret, m) == 1:
            return ret

## SET UP

In [ ]:
# UNKOWN
p = primes[-1]
q = primes[-2]
s = pick_random_seed(p * q)
print('These value are secret:\n\tp={0},\n\tq={1},\n\ts={2}'.format(p, q, s))

# KNOWN
a = 1
b = 1
n = 42
B = 100
print('These value are no more a secret:\n\ta={0},\n\tb={1},\n\tn={2},\n\tB={3}'.format(a, b, n, B))

# Construct `plainfile` and `cipherimage`

In [ ]:
plainfile = np.random.randint(0, 255, B, 'B').tobytes()
cipherimage = Application(p, q, s, a, b, n).encrypt(plainfile)

## Attack

In [ ]:
start = time()
# ENCRYPT
## TRANSFORMATION
plainbytes = transform.encryption_formatting(plainfile)
plainbytes = transform.pad_bytes(plainbytes)

# DECRYPT
## ARNOLD'S CAT MAP
plainimage = ACM(a, b, n).decrypt(cipherimage)

## TRANSFORMATION
ciphermatrix = transform.image_to_matrix(plainimage)
cipherbytes = transform.matrix_to_bytes(ciphermatrix)

# GET KEYSTREAM
keystream = np.zeros(plainbytes.size, np.dtype('B'))
i = 0
while i < len(plainbytes):
    keystream[i] = plainbytes[i] - cipherbytes[2 * i] + cipherbytes[2 * i + 1]
    i += 1

# SEARCHING FOR p, q, and s
i = 0
found = False
while i < len(primes) - 1 and not found:
    p_a = primes[i]
    j = i + 1
    while j < len(primes) and not found:
        q_a = primes[j]
        s_s = keystream[0]
        try:
            bbs = BBS(p_a, q_a, s_s)
            matched = True
            k = 1
            while k < keystream.size and matched:
                K = bbs.next()
                if keystream[k] != K:
                    matched = False
                else:
                    k += 1
            if k == keystream.size:
                found = True
        except ValidationError:
            j += 1
        j += 1
    i += 1

s_a = 0
m_a = p_a * q_a
found = False
while s_a < m_a and not found:
    if nt.mod_pow(s_a, 2, m_a) == s_s:
        found = True
    else:
        s_a += 1

print('Time required: {0}'.format(time() - start))
plainfile_f = Application(p_a, q_a, s_a, a, b, n).decrypt(cipherimage)
if plainfile == plainfile_f:
    print('We got the key')
    print(p_a, q_a, s_a)
else:
    print('Gone wrong')